### 1. 简单的示例 

<!-- ![image](docs/stream_warn.png) -->
<img src="docs/stream_warn.png" width="800" alt="stream warn">

In [ ]:
# 1.1 简单的示例

from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model([   "https://ultralytics.com/images/bus.jpg", 
                    "https://ultralytics.com/images/zidane.jpg"], stream=True)  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

### 2. 多种输入源

<!-- ![image](docs/source_type.png) -->
<img src="docs/source_type.png" width="800" alt="source_type">


In [ ]:
# 2.1 截屏目标检测

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define current screenshot as source
source = "screen"

# Run inference on the source
results = model(source, stream=True)  # list of Results objects

results[0].show()

In [ ]:
# 2.2 摄像头目标检测

import cv2
from ultralytics import YOLO
import time

# Load the YOLO model
model = YOLO("yolo11n.pt")

# Open the default camera (0 is usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("错误: 无法打开摄像头")
    exit()

print("摄像头已成功打开。按 'q' 键退出。")
window_name = "YOLO 摄像头目标检测"

try:
    # Loop through the camera frames
    while True:
        # Read a frame from the camera
        success, frame = cap.read()
        
        if success:
            # Run YOLO inference on the frame
            results = model(frame)
            
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            
            # Display the annotated frame
            cv2.imshow(window_name, annotated_frame)
            
            key = cv2.waitKey(30) & 0xFF
            if key == ord("q") or cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        else:
            print("错误: 无法读取摄像头画面")
            break

finally:
    cv2.destroyAllWindows()
    cap.release()
    time.sleep(3)
    print("摄像头已释放，窗口已关闭。")

### 3. 推理参数

<!-- ![image](docs/model_predict.png) -->

<img src="docs/model_predict.png" width="800" alt="model_predict">

In [ ]:
# 3.1 推理参数分类

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Run inference on 'bus.jpg' with arguments
results = model.predict(
                        # ---- 1. 输入配置与基础日志参数（定义数据源+控制运行日志）-------------------------------------------------------------------
                        source = "https://ultralytics.com/images/bus.jpg",  # 核心：指定推理数据源（图片路径/URL、视频、摄像头、目录等）【无固定默认值，需手动指定】
                        verbose = True,                                     # 默认值：True；辅助：控制终端是否打印详细推理日志（进度、结果统计等）
                        
                        # ---- 2. 数据预处理参数（对输入图像/视频进行预处理，适配模型输入）-------------------------------------------------------------------
                        imgsz = 320,                # 默认值：640；图像缩放：推理时的输入图像尺寸，平衡精度与速度
                        batch = 1,                  # 默认值：1；批量处理：指定推理批大小（仅对目录/视频/.txt文件有效），提升批量吞吐量
                        vid_stride = 1,	            # 默认值：1；视频跳帧：视频输入的帧步长，值越大跳帧越多，速度越快但时间分辨率降低
                        stream_buffer = False,      # 默认值：False；视频缓存：控制视频流帧的缓存策略（False丢弃旧帧适配实时，True排队帧避免遗漏）
                        rect = True,                # 默认值：True；矩形优化：（专属检测）矩形检测模式，减少图像填充冗余，优化预处理效率与检测精度
                        
                        # ---- 3. 模型推理与效率优化参数（控制模型推理的设备、速度、内存占用）-------------------------------------------------------------------
                        device = None,              # 默认值：None；设备指定：指定推理设备（cpu/cuda:0/0等），分配计算资源
                        half = False,               # 默认值：False；精度优化：启用FP16半精度推理，支持GPU的场景下提升速度，精度损失极小
                        compile = False,            # 默认值：False；编译优化：启用PyTorch 2.x torch.compile图形编译，优化推理速度（不支持则回退）
                        stream = False,             # 默认值：False；内存优化：启用流式处理，返回Results生成器，避免大批量/长视频加载占用过多内存
                        
                        # ---- 4. 检测结果筛选与后处理参数（对模型原始输出进行提纯，得到最终有效结果）-------------------------------------------------------------------
                        conf = 0.5,                 # 默认值：0.25；置信度过滤：设置最小置信度阈值，过滤低置信度无效检测结果
                        iou = 0.7,                  # 默认值：0.7；NMS阈值：设置非极大值抑制的IoU阈值，去除重叠度过高的重复检测框
                        max_det = 300,              # 默认值：300；最大检测数：限制单张图像/单帧视频的最大目标检测数量，避免密集场景输出过载
                        agnostic_nms = False,       # 默认值：False；类别无关NMS：合并不同类别间的重叠框，适配多类别目标重叠场景
                        classes = None,             # 默认值：None；类别筛选：按类别ID筛选结果，仅返回指定类别的检测目标
                        
                        # ---- 5. 结果输出与可视化参数（分为「持久化保存」和「实时窗口展示」，二者独立配置）-------------------------------------------------------------------
                        # 5.1 结果持久化保存（将结果存入本地文件，用于后续分析/归档）
                        save = True,                # 默认值：False（Python）/True（CLI）；核心保存开关：启用标注结果（图像/视频）的本地保存
                        project = None,             # 默认值：None；保存根目录：指定结果保存的项目根目录（仅save=True时有效）
                        name = None,                # 默认值：None；保存子目录：指定推理运行名称，在project下创建子目录区分结果（仅save=True时有效）
                        save_frames = False,        # 默认值：False；视频帧保存：处理视频时，将每帧标注结果单独保存为图像文件
                        save_txt = False,           # 默认值：False；文本结果保存：将检测结果以[class x_center y_center width height confidence]格式保存为.txt文件
                        save_conf = False,          # 默认值：False；置信度保存：配合save_txt=True，在txt文件中附带目标置信度分数
                        save_crop = False,          # 默认值：False；目标裁剪保存：将检测到的目标从原图中裁剪出来，单独保存为图像文件
                        
                        # 5.2 实时窗口可视化（仅推理时展示窗口，无持久化，用于调试/实时验证）
                        show = False,               # 默认值：False；窗口展示开关：启用弹出窗口，显示带标注的图像/视频帧
                        show_labels = True,         # 默认值：True；标签展示：在可视化窗口中显示目标类别标签
                        show_conf = True,           # 默认值：True；置信度展示：在类别标签旁显示目标检测置信度分数
                        show_boxes = True,          # 默认值：True；边界框展示：在可视化窗口中为目标绘制矩形边界框
                        line_width = None,          # 默认值：None；框线宽度：指定边界框线条宽度，None时自动适配图像尺寸
                        
                        # ---- 6. 高级功能扩展参数（非核心检测功能，用于调试/分割/下游任务）-------------------------------------------------------------------
                        visualize = False,          # 默认值：False；特征可视化：激活模型各层特征图的可视化与保存，用于调试和理解模型工作原理
                        augment = False,            # 默认值：False；测试时增强（TTA）：启用推理时的数据增强，提升检测鲁棒性，代价是降低推理速度
                        retina_masks = False,       # 默认值：False；高分辨率掩码：（专属分割）返回与原始图像尺寸匹配的高分辨率分割掩码
                        embed = None                # 默认值：None；特征提取：指定特征层，提取目标的特征向量/嵌入向量，用于聚类、相似性搜索等下游任务
                        )

for result in results:
    result.show()

### 4. 图像和视频格式

- Ultralytics 支持的图像格式：.bmp .dng .jpeg .jpg .mpo .png .tif .tiff .webp .pfm .HEIC;
- Ultralytics 支持的视频格式：.asf .avi .gif .m4v .mkv .mov .mp4 .mpeg .mpg .ts .wmv .webm。

### 5. 处理结果 Results

<img src="docs/result_porperty.png" width="800" alt="result_porperty"/>

<img src="docs/result_function.png" width="800" alt="result_function"/>

In [ ]:
# 5.1 边界框、掩码、关键点（Boxes、Masks、Keypoints）

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Run inference on an image
results = model("https://ultralytics.com/images/bus.jpg")  # results list

# View results
for r in results:
    print(r.boxes)      # Boxes 对象可用于索引、操作和将边界框转换为不同的格式。
    print(r.masks)      # Masks 对象可用于索引、操作和将掩码转换为分割。
    print(r.keypoints)  # Keypoints 对象可用于索引、操作和归一化坐标。
    print(r.probs)      # Probs 对象可用于索引、获取 top1 和 top5 分类的索引和分数。
    print(r.obb)        # OBB 对象可用于索引、操作和转换有向边界框为不同的格式。

In [ ]:
# 5.2 结果绘图
# 字段 plot() 方法在 Results 对象有助于通过将检测到的对象（例如边界框、掩码、关键点和概率）叠加到原始图像上来可视化预测。
# 此方法将带注释的图像作为 NumPy 数组返回，便于显示或保存。

from PIL import Image

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Run inference on 'bus.jpg'
results = model(["https://ultralytics.com/images/bus.jpg", 
                 "https://ultralytics.com/images/zidane.jpg"])  # results list

# Visualize the results
for i, r in enumerate(results):

    # Plot results image
    im_bgr = r.plot(
        conf = True,            # 默认值：True； 包括检测置信度分数。
        line_width = None,      # 默认值：None； 边界框的线条宽度。如果为None，则随图像大小缩放。
        font_size = None,       # 默认值：None； 文本字体大小。如果为None，则随图像大小缩放。
        font = 'Arial.ttf',     # 默认值：'Arial.ttf'； 文本注释的字体名称。
        pil = False,            # 默认值：False； 将图像作为 PIL 图像对象返回。
        img = None,             # 默认值：None； 用于绘制图像的替代图像。如果未提供，则使用原始图像。
        im_gpu = None,          # 默认值：None； 用于更快进行掩码绘制的 GPU 加速图像。形状：(1, 3, 640, 640)。
        kpt_radius = 5,         # 默认值：5； 绘制的关键点半径。
        kpt_line = True,        # 默认值：True； 用线条连接关键点。
        labels = True,          # 默认值：True； 在注释中包含类别标签。
        boxes = True,           # 默认值：True； 在图像上叠加边界框。
        masks = True,           # 默认值：True； 在图像上叠加掩码。
        probs = True,           # 默认值：True； 包含分类概率。
        show = False,           # 默认值：False； 使用默认图像查看器直接显示带注释的图像。
        save = False,           # 默认值：False； 将带注释的图像保存到指定的文件中。
        filename = None,        # 默认值：None； 如果指定了 ，则为保存带注释图像的文件的路径和名称。
        color_mode = 'class',   # 默认值：'class'； 指定颜色模式，例如“instance”或“class”。
        txt_color = (255, 255, 255) # 默认值：(255, 255, 255)； 用于边界框和图像分类标签的 RGB 文本颜色。
    )  # BGR-order numpy array

    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # r.show()
    im_rgb.show()

    # Save results to disk
    # r.save(filename=f"results{i}.jpg")
    # im_rgb.save(filename=f"results{i}.jpg")

### 6. 线程安全推理

当您在不同线程中并行运行多个 YOLO 模型时，确保推理过程中的线程安全至关重要。线程安全推理保证每个线程的预测都是隔离的，不会相互干扰，从而避免竞争条件，并确保输出的一致性和可靠性。

在多线程应用程序中使用 YOLO 模型时，为每个线程实例化单独的模型对象或采用线程局部存储以防止冲突非常重要：

In [ ]:
# 6.1 在每个线程内实例化一个模型，以实现线程安全推理：

from threading import Thread

from ultralytics import YOLO


def thread_safe_predict(model, image_path):
    """Performs thread-safe prediction on an image using a locally instantiated YOLO model."""
    model = YOLO(model)
    results = model.predict(image_path)

    for result in results:
        result.show()


# Starting threads that each have their own model instance
Thread(target=thread_safe_predict, args=("yolo11n.pt", "https://ultralytics.com/images/bus.jpg")).start()
Thread(target=thread_safe_predict, args=("yolo11n.pt", "https://ultralytics.com/images/zidane.jpg")).start()

### 7. 流媒体源 for-循环

这是一个使用OpenCV的Python脚本（cv2) 和 YOLO 运行视频帧推理的 python 脚本。此脚本假定您已安装必要的软件包 (opencv-python 和 ultralytics）。

In [ ]:
# 7.1 使用cv2加载视频，流式 for 循环

import cv2, time

from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11n.pt")

# Open the video file
video_path = "../video/mixkit-tokyo-street-night.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLO Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
time.sleep(3)

In [ ]:
# 7.2 流式视频检测（直接传视频文件给Yolo）

from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11n.pt")

# Open the video file
video_path = "../video/mixkit-people-snow.mp4"

results = model.predict(
    video_path,
    vid_stride = 50,
    # stream = True,
    show=True,
    # save=True
)


In [ ]:
# 7.3 流式视频检测（使用cv2加载控制视频流）

import cv2
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11n.pt")

# Open the video file (绝对路径更稳妥)
video_path = "../video/mixkit-people-building.mp4"
cap = cv2.VideoCapture(video_path)

# 关键初始化：帧计数器 + 上一次推理结果缓存
frame_count = 0
last_detection_result = None

# 循环读取每帧（保证画面连续）
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # 视频读取完毕，退出循环

    frame_count += 1  # 每读取一帧，计数器+1

    # 核心逻辑：每50帧执行一次真实推理，其余帧复用结果
    if frame_count % 3 == 1:  # 第1、51、101...帧执行推理（可改为==0，对应第50、100...帧）
        # 仅对关键帧执行YOLO推理，更新缓存结果
        results = model.predict(frame, verbose=False)  # verbose=False关闭冗余日志
        last_detection_result = results[0]
        # 基于最新推理结果生成标注帧
        annotated_frame = last_detection_result.plot()
    else:
        # 非关键帧：复用上一次的推理结果，保证画面连续有检测框
        if last_detection_result is not None:
            # 关键步骤：将当前帧赋值给缓存结果的原始帧属性，再重新绘制标注
            last_detection_result.orig_img = frame
            annotated_frame = last_detection_result.plot()
        else:
            # 若还未执行过推理（前50帧内），直接使用原始帧
            annotated_frame = frame

    # 显示连续的标注视频（画面无跳帧）
    cv2.imshow("YOLO Continuous Playback", annotated_frame)

    # 控制播放速度（可调整为1ms实现快速播放，不影响画面连续性）
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 释放资源
cap.release()
cv2.destroyAllWindows()